This is a quick tutorial on using OData v4 with Pyslet.  It roughly follows the basic examples here: http://www.odata.org/odata-services/

If you want to watch a video tutorial for this notebook there's one on YouTube here: http://www.youtube.com/watch?v=_HkKwK9ePcI

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
from pyslet.odata4.client import Client

The following little trick enables to to catch and print errors that happen when running cells.  (Sometimes we generate errors deliberately to illustrate a point.)

In [2]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def handle(line, cell):
    try:
        exec(cell)
    except Exception as e:
        msg = str(e)
        if msg:
            print(msg)
        else:
            print(type(e))

You can get the URL of the TripPin reference service from the OData website, it uses a redirect to a URL containing a session token enabling us to demonstrate data modification too, nice!  Pyslet also parses the metadata document when you create the Client object which it uses to enforce the correct typing and behaviours on data model objects.

In [3]:
svc = Client('http://services.odata.org/TripPinRESTierService')

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService HTTP/1.1'
INFO:root:Finished Response, status 302
INFO:root:Resending request to: http://services.odata.org/TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/
INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/ HTTP/1.1'
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/$metadata HTTP/1.1'
INFO:root:Finished Response, status 200
INFO:root:Service metadata format: application/xml


OData services expose an EntityContainer containing EntitySets (like SQL Tables) and Singletons bound to specific URLs in the service document.  You can open an EntitySet directly from the client object which is treated as a reference to the data service.

In [ ]:
people = svc.open('People')
print(type(people))
print(type(people.type_def))
print(type(people.type_def.item_type))
people.type_def.item_type.qname

Pyslet's OData support is dictionary-like.  An EntitySet is a collection of Entities each of which has a key.  It behaves like a mapping from the entity key to the entity value.  When you iterate a dictionary you get the keys... '

In [ ]:
ip = iter(people)
print(next(ip))
print(next(ip))
list(ip)

The EntitySetValue object caches information to minimise the number of requests it has to make to the data source.  When we access an entity by key we get the cached value, if available.  If we explicitly clear the cache we force the object to retrieve the data from the service again.

In [ ]:
russellwhyte = people['russellwhyte']
print(id(russellwhyte))
people.clear_cache()
russellwhyte = people['russellwhyte']
print(id(russellwhyte))

In [ ]:
airports = svc.open('Airports')
sfo = airports['KSFO']
print(sfo)
print(list(sfo.keys()))
print(type(sfo['Name']))
print(sfo['Name'].get_value())

In [ ]:
sfo['Name'].set_value('Mills Field Municipal Airport')
print(sfo['Name'].dirty)
print(sfo['Name'].get_value())
sfo['Name'].reload()
print(sfo['Name'].dirty)
print(sfo['Name'].get_value())

In [ ]:
print(type(sfo['Location']['Address']))
sfo['Location']['Address'].reload()
print(sfo['Location'])
sfo['Location'].reload()

In [ ]:
print(russellwhyte)
print(type(russellwhyte['AddressInfo']))
print(len(russellwhyte['AddressInfo']))
print(russellwhyte['AddressInfo'][0])
russellwhyte['AddressInfo'].reload()

To filter an EntitySet we just add a filter to our people object.  This effectively reduces the size of the dictionary to include only entities that match the filter.  The cache is cleared automatically.

In [ ]:
people.set_filter("FirstName eq 'Scott'")
[p for p in people]

Pyslet contains an expression parser so if you forget your OData syntax you'll get an error from Python and not from the remote service

In [ ]:
%%handle
people.set_filter("FirstName == 'Scott'")

In [ ]:
airports.set_filter("contains(Location/Address, 'San Francisco')")
[a for a in airports]

In [ ]:
people.set_filter("Gender eq Microsoft.OData.Service.Sample.TrippinInMemory.Models.PersonGender'Female'")
[p for p in people]

These filters reduce the number of entities in our entity set.  Another type of filtering involves reducing the number of properties returned for each entity.  By default, *all* properties are selected.  As soon as we select a specific property then only those we've explicitly selected are returned.

In [ ]:
airports.select("Name")
airports.select("IcaoCode")
airports.set_filter(None)
[a for a in airports]

In [ ]:
sfo = airports['KSFO']
list(sfo.keys())

The next example involves a navigation property.  By default, navigation properties are not expanded and don't appear in the property dictionary of the Entity, let's remove the filter from people and *expand* a navigation property.

In [ ]:
people.set_filter(None)
people.expand("Trips")
scottketchum = people['scottketchum']
scottketchum['Trips']

You might expect this to be an EntitySet, rather than a collection.  Afterall, this is a navigation property that relates a person to a collection of Trip *entities*...

In [ ]:
print(type(scottketchum['Trips'].type_def.item_type))
print(scottketchum['Trips'].type_def.item_type.qname)

The reason for this is beyond the scope of a basic tutorial but if you want to know the answer, take a closer look at the metadata description of the TripPin service.  Pay close attention to the definition of the People EntitySet!  Although we can't look up Trips by key we can still apply an ordering (or a filter) to the output.

In [ ]:
scottketchum['Trips'].set_orderby("EndsAt desc")
[str(t['EndsAt']) for t in scottketchum['Trips']]

In [ ]:
scottketchum['Trips'].set_orderby("EndsAt asc")
[str(t['EndsAt']) for t in scottketchum['Trips']]

If you're familiar with Pyslet's OData v2 support watch out for the handling of top/skip, we now treat them the same as any other filter, effectively reducing the size of the collection (or entity set) to include only those entities in the designated range.  They're always set together.

In [ ]:
people.collapse("Trips")
people.set_page(top=2)
[p for p in people]

In [ ]:
people.set_page(top=None, skip=18)
[p for p in people]

In [ ]:
people.set_page(top=None)
len(people)

Singletons are like EntitySets except that they contain only one entity.  You can use select and expand but you can't use filters or ordering.  They are not indexable, to retrieve the entity they contain you just *call* them. 

In [ ]:
whoami = svc.open('Me')
print(type(whoami))
whoami.expand('Friends')
me = whoami()
print(me['UserName'])

In [ ]:
fof = me['Friends']
fof.set_filter("Friends/any(f:f/FirstName eq 'Scott')")
[str(f['UserName']) for f in fof]

In [ ]:
people.expand('Trips')
russellwhyte = people['russellwhyte']
len(russellwhyte['Trips'])

You can control a filter or page option on expansion using an explicit xpath...

In [ ]:
people.set_page(top=1, xpath="Trips")
russellwhyte = people['russellwhyte']
len(russellwhyte['Trips'])

For a selection, you just use the full path to the desired property.  The expansion is done for you automatically, in the example below we use a clean people object to demonstrate.

In [ ]:
people = svc.open('People')
people.select("Trips/TripId")
people.select("Trips/Name")
russellwhyte = people['russellwhyte']
list(russellwhyte['Trips'][0].keys())

In [ ]:
people.set_filter("Name eq 'Trip in US'", xpath="Trips")
russellwhyte = people['russellwhyte']
[str(t['Name']) for t in russellwhyte['Trips']]

Creating entities is a multi-step process.  You create an entity object (using *new_item*) which is a transient value not yet bound to the data service.  It is also 'null' which means there are no properties!  Make the entity non-null by setting defaults, then set the values of the properties as desired and finally insert the new entity into the entity set.

In [ ]:
%%handle
people.collapse("Trips")
people['lewisblack']

In [ ]:
lewisblack = people.new_item()
print(lewisblack.is_null())
print(list(lewisblack.keys()))
lewisblack.set_defaults()
print(lewisblack.is_null())
print(lewisblack.type_def['FirstName'].nullable)
repr(lewisblack['FirstName'].get_value())

set_defaults only marks fields dirty if there is a default and the property has been updated as a result, otherwise it leaves the value *unchanged*.

In [ ]:
lewisblack['FirstName'].set_value("Lewis")
lewisblack['FirstName'].clean()
print(lewisblack['FirstName'].dirty)
lewisblack.set_defaults()
print(lewisblack['FirstName'])
print(lewisblack['FirstName'].dirty)

By chance the JSON dictionary format of the example is valid python so we can set all the properties in one go...

In [ ]:
lewisblack.set_value({
    "UserName":"lewisblack",
    "FirstName":"Lewis",
    "LastName":"Black",
    "Emails":[
        "lewisblack@example.com"
    ],
    "AddressInfo": [
    {
      "Address": "187 Suffolk Ln.",
      "City": {
        "Name": "Boise",
        "CountryRegion": "United States",
        "Region": "ID"
      }
    }
    ]
})
lewisblack['AddressInfo'][0]['City']['Name'].get_value()

In [ ]:
%%handle
people.insert(lewisblack)

What went wrong?  The trouble is that there are properties in the People entity that are non-nullable but that don't have defaults set so we *must* provide values for those fields.  Gender is an example.

In [ ]:
print(lewisblack['Gender'].is_null())
print(lewisblack.type_def['Gender'].nullable)

But in the example, Gender is not set so there must be some implicit default or computed value.  The way around the problem is to send only those properties that we've changed.  According to the specification it is OK to leave out properties if they can be computed by the service or are tied by referential constraints.  Also, services *may* add new nullable properties from time-to-time without breaking old clients so it's clearly OK to leave nullable values out.  Ideally, non-nullable values like Gender would be marked as being Computed so we would know it is OK to leave them out but this annotation is not used by TripPin.  The omit_clean option on insert allows us to override the default behaviour, effectively treating all clean values as either nullable, constrained or computable.

In [ ]:
lewisblack = people.new_item()
lewisblack.set_defaults()
lewisblack.set_value({
    "UserName":"lewisblack",
    "FirstName":"Lewis",
    "LastName":"Black",
    "Emails":[
        "lewisblack@example.com"
    ],
    "AddressInfo": [
    {
      "Address": "187 Suffolk Ln.",
      "City": {
        "Name": "Boise",
        "CountryRegion": "United States",
        "Region": "ID"
      }
    }
    ]
})
print(lewisblack['FirstName'].dirty)
print(lewisblack['Gender'].dirty)
people.insert(lewisblack, omit_clean=True)

In [ ]:
people.clear_cache()
lewisback = people['lewisblack']
lewisblack['AddressInfo'][0]['City']['Name'].get_value()

Out of curiosity, what was our computed Gender?

In [ ]:
lewisblack['Gender'].get_value()

Updating an entity works in a similar way except that PATCH semantics mean that only modified fields are sent to the data service anyway.  Changes are only made when you *commit* them, after which they are marked clean again.

In [ ]:
russellwhyte = people['russellwhyte']
russellwhyte['FirstName'].set_value("Mirs")
russellwhyte['LastName'].set_value("King")
russellwhyte.commit()
russellwhyte['FirstName'].dirty

In [ ]:
people.clear_cache()
russellwhyte = people['russellwhyte']
print("%s %s" % (russellwhyte['FirstName'], russellwhyte['LastName']))

Finally, when you want to remove an entity you just delete it from the dictionary in the normal way.

In [ ]:
del people['russellwhyte']
print(len(people))
print(people.get('russellwhyte', "Russell Whyte has been terminated"))

Unbound functions can be opened just like EntitySets and Singletons

In [4]:
func = svc.open('GetNearestAirport')
print(type(func))
print(type(func.type_def))
func.type_def.qname

<class 'pyslet.odata4.model.CallableValue'>
<class 'pyslet.odata4.model.Function'>


'Microsoft.OData.Service.Sample.TrippinInMemory.Models.GetNearestAirport'

The CallableValue object behaves like a dictionary of parameter values, you set them just like you would set property values of Entities.  You *call* a function simply by calling it.

In [6]:
func['lat'].set_value(33)
func['lon'].set_value(-11)
airport = func()
str(airport['Name'])

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'GET /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/GetNearestAirport(lat%3D33.0%2Clon%3D-11.0) HTTP/1.1'
INFO:root:Finished Response, status 200


'John F. Kennedy International Airport'

Bound functions are opened from the value they are bound to using the get_callable method which takes the qualified name of the desired function as an argument.  Unfortunately we have to revist the problem we raised earlier with Trips being a CollectionValue instead of the more powerful EntitySetValue.  To access the Trip with TripId 0 we'd like to look up the key with trips[0] but that only does index lookup so returns the first Trip in the collection (which could have any TripId).  CollectionValue has a set_key_filter method to get around the problem, it filters the collection by a specific key value so that when we get the first entity in the list it is sure to be the entity we wanted.  This avoids iterating the whole collection scanning for ['TripId'].get_value() == 0.

In [10]:
people = svc.open('People')
people.expand('Trips')
russellwhyte = people['russellwhyte']
trips = russellwhyte['Trips']
trips.set_key_filter(0)
trip = trips[0]
f = trip.get_callable("Microsoft.OData.Service.Sample.TrippinInMemory.Models.GetInvolvedPeople")
ipeople = f()
[str(ip['UserName']) for ip in ipeople]

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/People('russellwhyte')?%24expand=Trips HTTP/1.1"
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/People('russellwhyte')/Trips?%24filter=TripId%20eq%200 HTTP/1.1"
INFO:root:Finished Response, status 200
INFO:root:Sending request to b'services.odata.org'
INFO:root:b"GET /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/People('russellwhyte')/Trips(0)/Microsoft.OData.Service.Sample.TrippinInMemory.Models.GetInvolvedPeople() HTTP/1.1"
INFO:root:Finished Response, status 200


['russellwhyte', 'scottketchum']

There is almost no difference between Functions and Actions in this API.  Bear in mind that an Action *may* return None indicating that there is no return value, whereas a Function *must* have a return value (even if it is null).

In [12]:
f = svc.open('ResetDataSource')
repr(f())

INFO:root:Sending request to b'services.odata.org'
INFO:root:b'POST /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/ResetDataSource HTTP/1.1'
INFO:root:Finished Response, status 204


'None'

In [13]:
action = russellwhyte.get_callable("Microsoft.OData.Service.Sample.TrippinInMemory.Models.ShareTrip")
action['userName'].set_value('scottketchum')
action['tripId'].set_value(0)
repr(action())

INFO:root:Sending request to b'services.odata.org'
INFO:root:b"POST /TripPinRESTierService/(S(c3qyeyrj5dfvdttasunydd3y))/People('russellwhyte')/Microsoft.OData.Service.Sample.TrippinInMemory.Models.ShareTrip HTTP/1.1"
INFO:root:Finished Response, status 204


'None'